In [2]:
from media_impact_monitor.impact import get_impact, ImpactSearch, time_series_regression
from media_impact_monitor.events import get_events, EventSearch
from datetime import date
from media_impact_monitor.trend import get_trend, TrendSearch
import pandas as pd
from great_tables import GT
from IPython.display import display, HTML

organizers = ["Fridays for Future", "Extinction Rebellion", "Greenpeace"]
organizers = [organizers[0]]
start_date = date(2022, 1, 1)
end_date = date(2024, 1, 1)
rows = []
for method in ["time_series_regression"]:
    trend = get_trend(
        TrendSearch(
            trend_type="keywords",
            media_source="news_print",
            topic="climate_change",
            start_date=start_date,
            end_date=end_date,
        ),
        as_json=False,
    )["climate activism"].rename("count")
    for organizer in organizers:
        events = get_events(
            EventSearch(
                source="acled",
                organizers=[organizer],
                start_date=start_date,
                end_date=end_date,
            )
        )
        aggregation = "daily"
        match method:
            case "time_series_regression":
                for lags, ewmas in [
                    ([], []),
                    ([1], []),
                    ([1, 2, 3, 4, 5], []),
                    ([4, 5, 6, 7, 8], []),
                    ([], [1, 7, 28, 84]),
                ]:
                    ts, limitations = time_series_regression(
                        events=events,
                        article_counts=trend,
                        aggregation=aggregation,
                        lags=lags,
                    )
                    row = dict(
                        organizer=organizer,
                        method=method,
                        aggregation=aggregation,
                        lags=lags,
                        ewmas=ewmas,
                    )
                    row["rmse"] = ts["rmse"].loc[7]
                    row["rmse_std"] = ts["rmse_std"].loc[7]
                    row["ame"] = ts["ame"].loc[7]
                    row["ame_std"] = ts["ame_std"].loc[7]
                    row["time_series"] = " ".join(ts["mean"].values.astype(str))
                    rows.append(row)
df = pd.DataFrame(rows)
for organizer in organizers:
    df_ = df[df["organizer"] == organizer].drop(columns="organizer")
    html = (
        GT(df_)
        .fmt_number(columns=["rmse", "rmse_std", "ame", "ame_std"], decimals=2)
        .fmt_nanoplot(
            columns="time_series",
        )  # restriction in great_tables, only supports one nanoplot column
        .render("html")
    )
    display(HTML(f"<h1>{organizer}</h1>"))
    display(HTML(html))
